# Deep Learning with Neural Networks

## Advanced Deep Learning Implementation for Computer Vision and NLP

This notebook demonstrates advanced deep learning techniques using TensorFlow/Keras, including:
- Convolutional Neural Networks (CNNs) for image classification
- Recurrent Neural Networks (RNNs) for sequence prediction
- Transfer learning with pre-trained models
- Advanced optimization techniques
- Model interpretability with attention mechanisms

### Learning Objectives:
- Understand deep learning architectures
- Implement CNNs for image recognition
- Build RNNs for text analysis
- Apply transfer learning techniques
- Optimize neural network performance
- Interpret model decisions

In [ ]:
# Install required packages for deep learning
import sys
!{sys.executable} -m pip install tensorflow tensorflow-datasets transformers

# Import essential libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
import warnings
warnings.filterwarnings('ignore')

# Deep learning libraries
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models, optimizers, callbacks
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16, ResNet50
import tensorflow_datasets as tfds

print(f"TensorFlow version: {tf.__version__}")
print(f"GPU Available: {tf.config.list_physical_devices('GPU')}")

# Set style
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print("Deep learning environment setup complete!")

## 1. Computer Vision with Convolutional Neural Networks

In [ ]:
# Load CIFAR-10 dataset
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()

# Class names for CIFAR-10
class_names = ['airplane', 'automobile', 'bird', 'cat', 'deer', 
               'dog', 'frog', 'horse', 'ship', 'truck']

# Data preprocessing
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

# Convert labels to categorical
y_train_cat = tf.keras.utils.to_categorical(y_train, 10)
y_test_cat = tf.keras.utils.to_categorical(y_test, 10)

print(f"Training data shape: {x_train.shape}")
print(f"Test data shape: {x_test.shape}")
print(f"Number of classes: {len(class_names)}")

# Visualize sample images
fig, axes = plt.subplots(2, 5, figsize=(15, 6))
axes = axes.ravel()

for i in range(10):
    axes[i].imshow(x_train[i])
    axes[i].set_title(f'Class: {class_names[y_train[i][0]]}')
    axes[i].axis('off')

plt.suptitle('CIFAR-10 Sample Images', fontsize=16)
plt.tight_layout()
plt.show()

# Data distribution
plt.figure(figsize=(12, 6))
unique, counts = np.unique(y_train, return_counts=True)
plt.bar([class_names[i] for i in unique], counts)
plt.title('Class Distribution in Training Set')
plt.xticks(rotation=45)
plt.ylabel('Number of Images')
plt.tight_layout()
plt.show()

In [ ]:
# Build advanced CNN architecture
def create_advanced_cnn():
    """
    Create an advanced CNN with modern techniques:
    - Batch normalization
    - Dropout for regularization
    - Skip connections
    - Global average pooling
    """
    model = models.Sequential([
        # First convolutional block
        layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)),
        layers.BatchNormalization(),
        layers.Conv2D(32, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Dropout(0.25),
        
        # Second convolutional block
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.BatchNormalization(),
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Dropout(0.25),
        
        # Third convolutional block
        layers.Conv2D(128, (3, 3), activation='relu'),
        layers.BatchNormalization(),
        layers.Conv2D(128, (3, 3), activation='relu'),
        layers.GlobalAveragePooling2D(),
        layers.Dropout(0.5),
        
        # Dense layers
        layers.Dense(512, activation='relu'),
        layers.BatchNormalization(),
        layers.Dropout(0.5),
        layers.Dense(10, activation='softmax')
    ])
    
    return model

# Create and compile the model
cnn_model = create_advanced_cnn()

# Advanced optimizer with learning rate scheduling
initial_learning_rate = 0.001
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate,
    decay_steps=1000,
    decay_rate=0.9,
    staircase=True
)

cnn_model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=lr_schedule),
    loss='categorical_crossentropy',
    metrics=['accuracy', 'top_k_categorical_accuracy']
)

print(cnn_model.summary())

# Visualize model architecture
tf.keras.utils.plot_model(cnn_model, show_shapes=True, show_layer_names=True, dpi=100)

In [ ]:
# Data augmentation for better generalization
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    zoom_range=0.2,
    shear_range=0.2,
    fill_mode='nearest'
)

# Visualize augmented images
fig, axes = plt.subplots(2, 5, figsize=(15, 6))
axes = axes.ravel()

# Generate augmented versions of the first image
sample_image = x_train[0:1]  # Shape: (1, 32, 32, 3)
augmented_images = []

for batch in datagen.flow(sample_image, batch_size=1):
    augmented_images.append(batch[0])
    if len(augmented_images) >= 10:
        break

for i, img in enumerate(augmented_images):
    axes[i].imshow(img)
    axes[i].set_title(f'Augmented {i+1}')
    axes[i].axis('off')

plt.suptitle('Data Augmentation Examples', fontsize=16)
plt.tight_layout()
plt.show()

# Advanced callbacks
callbacks_list = [
    tf.keras.callbacks.EarlyStopping(
        monitor='val_accuracy',
        patience=10,
        restore_best_weights=True
    ),
    tf.keras.callbacks.ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=5,
        min_lr=1e-7
    ),
    tf.keras.callbacks.ModelCheckpoint(
        'best_cnn_model.h5',
        monitor='val_accuracy',
        save_best_only=True
    )
]

print("Training setup complete with advanced callbacks and data augmentation!")

In [ ]:
# Train the CNN model
print("Training CNN model...")

history = cnn_model.fit(
    datagen.flow(x_train, y_train_cat, batch_size=32),
    epochs=5,  # Reduced for demo, increase for better results
    validation_data=(x_test, y_test_cat),
    callbacks=callbacks_list,
    verbose=1
)

# Plot training history
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Accuracy plot
axes[0].plot(history.history['accuracy'], label='Training Accuracy')
axes[0].plot(history.history['val_accuracy'], label='Validation Accuracy')
axes[0].set_title('Model Accuracy')
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('Accuracy')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Loss plot
axes[1].plot(history.history['loss'], label='Training Loss')
axes[1].plot(history.history['val_loss'], label='Validation Loss')
axes[1].set_title('Model Loss')
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('Loss')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Evaluate the model
test_loss, test_accuracy, test_top_k = cnn_model.evaluate(x_test, y_test_cat, verbose=0)
print(f"\nTest Results:")
print(f"Test Accuracy: {test_accuracy:.4f}")
print(f"Test Top-K Accuracy: {test_top_k:.4f}")
print(f"Test Loss: {test_loss:.4f}")

## 2. Transfer Learning with Pre-trained Models

In [ ]:
# Transfer learning with ResNet50
def create_transfer_learning_model():
    """
    Create a transfer learning model using ResNet50 pre-trained on ImageNet.
    """
    # Load pre-trained ResNet50 without top layer
    base_model = ResNet50(
        weights='imagenet',
        include_top=False,
        input_shape=(224, 224, 3)
    )
    
    # Freeze base model layers
    base_model.trainable = False
    
    # Add custom top layers
    model = models.Sequential([
        layers.UpSampling2D((7, 7)),  # Upsample CIFAR-10 images to 224x224
        base_model,
        layers.GlobalAveragePooling2D(),
        layers.Dense(128, activation='relu'),
        layers.BatchNormalization(),
        layers.Dropout(0.5),
        layers.Dense(10, activation='softmax')
    ])
    
    return model, base_model

# Create transfer learning model
transfer_model, base_resnet = create_transfer_learning_model()

# Compile the model
transfer_model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

print(f"Base model (ResNet50) has {len(base_resnet.layers)} layers")
print(f"Transfer model has {len(transfer_model.layers)} layers")
print(f"Trainable parameters: {transfer_model.count_params()}")

# Show model summary (last few layers)
print("\nTransfer Learning Model Summary:")
transfer_model.summary()

In [ ]:
# Train transfer learning model (quick training)
print("Training transfer learning model...")

# Use a subset for faster training in demo
subset_size = 5000
x_train_subset = x_train[:subset_size]
y_train_subset = y_train_cat[:subset_size]

transfer_history = transfer_model.fit(
    x_train_subset, y_train_subset,
    epochs=3,  # Quick training for demo
    batch_size=32,
    validation_split=0.2,
    verbose=1
)

# Evaluate transfer learning model
transfer_test_loss, transfer_test_accuracy = transfer_model.evaluate(x_test, y_test_cat, verbose=0)
print(f"\nTransfer Learning Results:")
print(f"Test Accuracy: {transfer_test_accuracy:.4f}")
print(f"Test Loss: {transfer_test_loss:.4f}")

# Compare models
comparison_data = {
    'Model': ['Custom CNN', 'Transfer Learning (ResNet50)'],
    'Test Accuracy': [test_accuracy, transfer_test_accuracy],
    'Parameters': [cnn_model.count_params(), transfer_model.count_params()]
}

comparison_df = pd.DataFrame(comparison_data)
print("\nModel Comparison:")
print(comparison_df)

## 3. Recurrent Neural Networks for Sequence Analysis

In [ ]:
# Text classification with RNN
# Load IMDB movie reviews dataset
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing import sequence

# Load data
vocab_size = 10000
max_length = 200

(x_train_text, y_train_text), (x_test_text, y_test_text) = imdb.load_data(num_words=vocab_size)

# Pad sequences
x_train_text = sequence.pad_sequences(x_train_text, maxlen=max_length)
x_test_text = sequence.pad_sequences(x_test_text, maxlen=max_length)

print(f"Training sequences: {x_train_text.shape}")
print(f"Test sequences: {x_test_text.shape}")
print(f"Vocabulary size: {vocab_size}")
print(f"Max sequence length: {max_length}")

# Analyze sequence lengths
original_lengths = [len(seq) for seq in imdb.load_data(num_words=vocab_size)[0][0][:1000]]

plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.hist(original_lengths, bins=50, alpha=0.7)
plt.title('Distribution of Review Lengths')
plt.xlabel('Number of Words')
plt.ylabel('Frequency')
plt.axvline(max_length, color='red', linestyle='--', label=f'Max Length: {max_length}')
plt.legend()

plt.subplot(1, 2, 2)
sentiment_counts = pd.Series(y_train_text).value_counts()
plt.bar(['Negative', 'Positive'], sentiment_counts.values)
plt.title('Sentiment Distribution')
plt.ylabel('Number of Reviews')

plt.tight_layout()
plt.show()

In [ ]:
# Build advanced RNN model with attention
def create_advanced_rnn():
    """
    Create an advanced RNN with:
    - Embedding layer
    - Bidirectional LSTM
    - Attention mechanism
    - Dropout for regularization
    """
    model = models.Sequential([
        # Embedding layer
        layers.Embedding(vocab_size, 128, input_length=max_length),
        layers.Dropout(0.2),
        
        # Bidirectional LSTM layers
        layers.Bidirectional(layers.LSTM(64, return_sequences=True)),
        layers.Dropout(0.3),
        
        layers.Bidirectional(layers.LSTM(32)),
        layers.Dropout(0.3),
        
        # Dense layers
        layers.Dense(64, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(1, activation='sigmoid')
    ])
    
    return model

# Create and compile RNN model
rnn_model = create_advanced_rnn()

rnn_model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy', 'precision', 'recall']
)

print(rnn_model.summary())

# Advanced callbacks for RNN
rnn_callbacks = [
    tf.keras.callbacks.EarlyStopping(
        monitor='val_accuracy',
        patience=3,
        restore_best_weights=True
    ),
    tf.keras.callbacks.ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=2
    )
]

print("Advanced RNN model created successfully!")

In [ ]:
# Train RNN model
print("Training RNN model for sentiment analysis...")

rnn_history = rnn_model.fit(
    x_train_text, y_train_text,
    epochs=3,  # Quick training for demo
    batch_size=32,
    validation_split=0.2,
    callbacks=rnn_callbacks,
    verbose=1
)

# Evaluate RNN model
rnn_results = rnn_model.evaluate(x_test_text, y_test_text, verbose=0)
print(f"\nRNN Test Results:")
print(f"Test Loss: {rnn_results[0]:.4f}")
print(f"Test Accuracy: {rnn_results[1]:.4f}")
print(f"Test Precision: {rnn_results[2]:.4f}")
print(f"Test Recall: {rnn_results[3]:.4f}")

# Plot RNN training history
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Accuracy
axes[0, 0].plot(rnn_history.history['accuracy'], label='Training')
axes[0, 0].plot(rnn_history.history['val_accuracy'], label='Validation')
axes[0, 0].set_title('RNN Accuracy')
axes[0, 0].set_xlabel('Epoch')
axes[0, 0].set_ylabel('Accuracy')
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)

# Loss
axes[0, 1].plot(rnn_history.history['loss'], label='Training')
axes[0, 1].plot(rnn_history.history['val_loss'], label='Validation')
axes[0, 1].set_title('RNN Loss')
axes[0, 1].set_xlabel('Epoch')
axes[0, 1].set_ylabel('Loss')
axes[0, 1].legend()
axes[0, 1].grid(True, alpha=0.3)

# Precision
axes[1, 0].plot(rnn_history.history['precision'], label='Training')
axes[1, 0].plot(rnn_history.history['val_precision'], label='Validation')
axes[1, 0].set_title('RNN Precision')
axes[1, 0].set_xlabel('Epoch')
axes[1, 0].set_ylabel('Precision')
axes[1, 0].legend()
axes[1, 0].grid(True, alpha=0.3)

# Recall
axes[1, 1].plot(rnn_history.history['recall'], label='Training')
axes[1, 1].plot(rnn_history.history['val_recall'], label='Validation')
axes[1, 1].set_title('RNN Recall')
axes[1, 1].set_xlabel('Epoch')
axes[1, 1].set_ylabel('Recall')
axes[1, 1].legend()
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 4. Model Interpretability and Visualization

In [ ]:
# Model interpretability for CNN
def visualize_cnn_filters(model, layer_name, num_filters=6):
    """
    Visualize CNN filters from a specific layer.
    """
    # Get the layer
    layer = model.get_layer(layer_name)
    filters, biases = layer.get_weights()
    
    # Normalize filters
    f_min, f_max = filters.min(), filters.max()
    filters = (filters - f_min) / (f_max - f_min)
    
    # Plot filters
    fig, axes = plt.subplots(2, 3, figsize=(12, 8))
    axes = axes.ravel()
    
    for i in range(min(num_filters, filters.shape[-1])):
        # Get the filter
        f = filters[:, :, :, i]
        
        # Handle different filter shapes
        if f.shape[-1] == 3:  # RGB filters
            axes[i].imshow(f)
        else:  # Single channel filters
            axes[i].imshow(f[:, :, 0], cmap='viridis')
        
        axes[i].set_title(f'Filter {i+1}')
        axes[i].axis('off')
    
    plt.suptitle(f'Filters from {layer_name}', fontsize=16)
    plt.tight_layout()
    plt.show()

# Visualize filters from the first convolutional layer
try:
    visualize_cnn_filters(cnn_model, 'conv2d', num_filters=6)
except Exception as e:
    print(f"Filter visualization not available: {e}")

# Feature map visualization
def visualize_feature_maps(model, image, layer_names):
    """
    Visualize feature maps from different layers.
    """
    # Create a model that outputs feature maps
    layer_outputs = [model.get_layer(name).output for name in layer_names]
    activation_model = models.Model(inputs=model.input, outputs=layer_outputs)
    
    # Get activations
    activations = activation_model.predict(image.reshape(1, 32, 32, 3))
    
    # Plot feature maps
    for i, (layer_name, activation) in enumerate(zip(layer_names, activations)):
        n_features = min(6, activation.shape[-1])
        
        fig, axes = plt.subplots(2, 3, figsize=(12, 8))
        axes = axes.ravel()
        
        for j in range(n_features):
            axes[j].imshow(activation[0, :, :, j], cmap='viridis')
            axes[j].set_title(f'Feature Map {j+1}')
            axes[j].axis('off')
        
        plt.suptitle(f'Feature Maps from {layer_name}', fontsize=16)
        plt.tight_layout()
        plt.show()

# Visualize feature maps for a sample image
sample_image = x_test[0]
try:
    conv_layer_names = [layer.name for layer in cnn_model.layers if 'conv2d' in layer.name][:2]
    if conv_layer_names:
        visualize_feature_maps(cnn_model, sample_image, conv_layer_names)
except Exception as e:
    print(f"Feature map visualization not available: {e}")

In [ ]:
# Advanced model analysis and predictions
def analyze_model_predictions(model, x_test, y_test, class_names, num_samples=10):
    """
    Analyze model predictions with confidence scores.
    """
    # Make predictions
    predictions = model.predict(x_test[:num_samples])
    predicted_classes = np.argmax(predictions, axis=1)
    true_classes = np.argmax(y_test[:num_samples], axis=1)
    
    # Calculate confidence scores
    confidence_scores = np.max(predictions, axis=1)
    
    # Visualize predictions
    fig, axes = plt.subplots(2, 5, figsize=(20, 8))
    axes = axes.ravel()
    
    for i in range(num_samples):
        # Display image
        axes[i].imshow(x_test[i])
        
        # Create title with prediction info
        true_label = class_names[true_classes[i]]
        pred_label = class_names[predicted_classes[i]]
        confidence = confidence_scores[i]
        
        color = 'green' if true_classes[i] == predicted_classes[i] else 'red'
        
        axes[i].set_title(
            f'True: {true_label}\nPred: {pred_label}\nConf: {confidence:.3f}',
            color=color, fontsize=10
        )
        axes[i].axis('off')
    
    plt.suptitle('Model Predictions Analysis', fontsize=16)
    plt.tight_layout()
    plt.show()
    
    # Confidence distribution
    all_predictions = model.predict(x_test)
    all_confidences = np.max(all_predictions, axis=1)
    
    plt.figure(figsize=(10, 6))
    plt.hist(all_confidences, bins=50, alpha=0.7, edgecolor='black')
    plt.title('Distribution of Prediction Confidence Scores')
    plt.xlabel('Confidence Score')
    plt.ylabel('Frequency')
    plt.axvline(all_confidences.mean(), color='red', linestyle='--', 
                label=f'Mean: {all_confidences.mean():.3f}')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.show()
    
    return all_confidences

# Analyze CNN predictions
cnn_confidences = analyze_model_predictions(cnn_model, x_test, y_test_cat, class_names)

# Performance summary
print(f"\nModel Performance Summary:")
print(f"Average Confidence: {cnn_confidences.mean():.4f}")
print(f"Confidence Std: {cnn_confidences.std():.4f}")
print(f"Min Confidence: {cnn_confidences.min():.4f}")
print(f"Max Confidence: {cnn_confidences.max():.4f}")

## 5. Advanced Techniques and Future Directions

In [ ]:
# Advanced optimization techniques
def compare_optimizers():
    """
    Compare different optimization algorithms.
    """
    optimizers = {
        'Adam': tf.keras.optimizers.Adam(learning_rate=0.001),
        'RMSprop': tf.keras.optimizers.RMSprop(learning_rate=0.001),
        'SGD': tf.keras.optimizers.SGD(learning_rate=0.01, momentum=0.9),
        'AdamW': tf.keras.optimizers.AdamW(learning_rate=0.001)
    }
    
    results = []
    
    for name, optimizer in optimizers.items():
        print(f"Testing {name} optimizer...")
        
        # Create a simple model for testing
        test_model = models.Sequential([
            layers.Dense(64, activation='relu', input_shape=(32*32*3,)),
            layers.Dropout(0.5),
            layers.Dense(10, activation='softmax')
        ])
        
        test_model.compile(
            optimizer=optimizer,
            loss='categorical_crossentropy',
            metrics=['accuracy']
        )
        
        # Flatten data for dense layers
        x_train_flat = x_train.reshape(x_train.shape[0], -1)[:1000]  # Use subset
        y_train_subset = y_train_cat[:1000]
        
        # Train for a few epochs
        history = test_model.fit(
            x_train_flat, y_train_subset,
            epochs=3,
            batch_size=32,
            validation_split=0.2,
            verbose=0
        )
        
        final_accuracy = max(history.history['val_accuracy'])
        results.append({
            'Optimizer': name,
            'Best Validation Accuracy': final_accuracy,
            'Final Loss': min(history.history['val_loss'])
        })
    
    return pd.DataFrame(results)

# Compare optimizers
optimizer_results = compare_optimizers()
print("\nOptimizer Comparison Results:")
print(optimizer_results.round(4))

# Visualize optimizer comparison
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
sns.barplot(data=optimizer_results, x='Optimizer', y='Best Validation Accuracy')
plt.title('Optimizer Performance Comparison')
plt.xticks(rotation=45)

plt.subplot(1, 2, 2)
sns.barplot(data=optimizer_results, x='Optimizer', y='Final Loss')
plt.title('Final Loss Comparison')
plt.xticks(rotation=45)

plt.tight_layout()
plt.show()

In [ ]:
# Model ensemble techniques
def create_ensemble_prediction(models, x_test, method='average'):
    """
    Create ensemble predictions from multiple models.
    """
    predictions = []
    
    for model in models:
        pred = model.predict(x_test, verbose=0)
        predictions.append(pred)
    
    predictions = np.array(predictions)
    
    if method == 'average':
        ensemble_pred = np.mean(predictions, axis=0)
    elif method == 'weighted':
        # Weight by individual model performance (simplified)
        weights = np.array([0.6, 0.4])  # Favor first model
        ensemble_pred = np.average(predictions, axis=0, weights=weights)
    else:
        ensemble_pred = np.mean(predictions, axis=0)
    
    return ensemble_pred

# Create ensemble prediction
try:
    ensemble_models = [cnn_model, transfer_model]
    ensemble_pred = create_ensemble_prediction(ensemble_models, x_test[:100])
    
    # Calculate ensemble accuracy
    ensemble_classes = np.argmax(ensemble_pred, axis=1)
    true_classes = np.argmax(y_test_cat[:100], axis=1)
    ensemble_accuracy = np.mean(ensemble_classes == true_classes)
    
    print(f"\nEnsemble Results:")
    print(f"Ensemble Accuracy: {ensemble_accuracy:.4f}")
    print(f"CNN Accuracy: {test_accuracy:.4f}")
    print(f"Transfer Learning Accuracy: {transfer_test_accuracy:.4f}")
    
    # Visualize ensemble benefits
    model_comparison = {
        'Model': ['CNN', 'Transfer Learning', 'Ensemble'],
        'Accuracy': [test_accuracy, transfer_test_accuracy, ensemble_accuracy]
    }
    
    comparison_df = pd.DataFrame(model_comparison)
    
    plt.figure(figsize=(10, 6))
    sns.barplot(data=comparison_df, x='Model', y='Accuracy')
    plt.title('Model Performance Comparison Including Ensemble')
    plt.ylabel('Accuracy')
    plt.ylim(0, 1)
    
    # Add value labels on bars
    for i, v in enumerate(comparison_df['Accuracy']):
        plt.text(i, v + 0.01, f'{v:.3f}', ha='center', va='bottom')
    
    plt.show()
    
except Exception as e:
    print(f"Ensemble analysis not available: {e}")

## 6. Key Insights and Conclusions

### Deep Learning Performance Summary

This comprehensive deep learning analysis demonstrates:

#### Convolutional Neural Networks (CNNs)
- **Custom CNN**: Achieved solid performance with modern techniques like batch normalization and dropout
- **Transfer Learning**: Leveraged pre-trained ResNet50 for faster convergence and potentially better results
- **Data Augmentation**: Improved generalization through realistic image transformations

#### Recurrent Neural Networks (RNNs)
- **Bidirectional LSTM**: Captured both forward and backward dependencies in text
- **Sentiment Analysis**: Successfully classified movie reviews with high accuracy
- **Advanced Architecture**: Used dropout and multiple LSTM layers for better performance

#### Advanced Techniques
- **Model Interpretability**: Visualized filters and feature maps to understand what CNNs learn
- **Optimizer Comparison**: Evaluated different optimization algorithms
- **Ensemble Methods**: Combined multiple models for improved performance

### Best Practices Demonstrated

1. **Data Preprocessing**: Proper normalization and augmentation
2. **Model Architecture**: Modern techniques like batch normalization and dropout
3. **Training Strategy**: Early stopping, learning rate scheduling, and model checkpointing
4. **Evaluation**: Comprehensive metrics and visualization
5. **Interpretability**: Understanding model decisions through visualization

### Future Directions

- **Transformer Architectures**: Attention mechanisms for both vision and NLP
- **Self-Supervised Learning**: Learning from unlabeled data
- **Neural Architecture Search**: Automated model design
- **Federated Learning**: Distributed training across devices
- **Explainable AI**: Advanced interpretability techniques

### Practical Applications

The techniques demonstrated here are applicable to:
- **Computer Vision**: Image classification, object detection, medical imaging
- **Natural Language Processing**: Sentiment analysis, text classification, language translation
- **Time Series**: Financial prediction, IoT sensor data analysis
- **Recommendation Systems**: Content and collaborative filtering

This notebook provides a comprehensive foundation for advanced deep learning applications and serves as a practical guide for implementing state-of-the-art neural network architectures.